# Simulación de ventas un carrito de helados

El dataframe tendrá valores del verano de 2019, del 2019-07-01 al 2019-08-30, se venden helados de tres sabores, que pueden variar su precio cada lunes de acuerdo a las condiciones de temperatura, los consumidores consumen más o menos helado de acuerdo con la variación de la temperatura base (25°) y el precio "base" ($50 por bola), simularemos la elasticidad de la demanda. Todas las bolas de helado cuestan igual.

El dataframe contiene los siguientes datos:

|campo              |significado y notas                                                |
|-------------------|-------------------------------------------------------------------|
|fecha              |Fecha de venta                                                     |
|localidad          |En que localidad se realizó la venta (alameda, escuela, playa)     |
|fresa              |Número de bolas de fresa que se vendieron                          |
|vainilla           |                                                                   |
|chocolate          |                                                                   |
|temperatura        |Temperatura del día (25°C como base)                               |
|publicidad         |Costo de la publicidad, de lunes a sábado volanteo, lunes en radio |
|precio             |Precio de la bola, se modifica los lunes con base en la temperatura|


In [ ]:
import pandas as pd
import random

random.seed(1234)

## Parámetros base

Las variables se modificarán de forma aleatoria conforme a estos parámetros que se definen aquí

In [ ]:
precio_base      = 50
temperatura_base = 25
precio           = 50
temperatura      = 25

## Localidades

El carrito de helados de lunes a jueves [0..3] únicamente se puede poner en la alameda, o afuera de la escuela, los viernes puede ser en la alameda, la escuela o la playa, los sábados solo en la alameda o la playa, y los domingos únicamente en la playa

Cada localidad tiene un cierto premium mínimo y máximo, el cual implica un riesgo. Las ventas en la zona de oficinas (la alameda) son más estables que en la playa, donde en un día soleado se puede vender hasta 30% más, pero si el día es malo, y nadie va a la playa, se puede vender 30% menos.

In [ ]:
localidad = {
    0: ["alameda", "escuela"],
    1: ["alameda", "escuela"],
    2: ["alameda", "escuela"],
    3: ["alameda", "escuela"],
    4: ["alameda", "escuela", "playa"],
    5: ["alameda", "playa"],
    6: ["playa"],    
}

premium = {
    "alameda": {'min': 0.95, 'max': 1.05},
    "escuela": {'min': 0.95, 'max': 1.12},
    "playa"  : {'min': 0.70, 'max': 1.30},
}

## Helado

* Se establece una venta promedio de 100 helados por cada sabor (fresa, vainilla y chocolate)
* Se define una función que hace que el consumo varie de acuerdo a la temperatura, a menor temperatura, menor consumo y a mayor temperatura, mayor consumo
* Se añade un poco de ruido a la función, a través de números aleatorios, indicando la variabilidad en las preferencias que tienen los consumidores por cada sabor

In [ ]:
# Función de variación de acuerdo a la temperatura
#
def venta(temp):
    return (((temp - temperatura_base)/(temp*1.2)) ** 3) * 40

# Variabilidad en las preferencias de los clientes (porcentual)
#
sabores = {
    'fresa'     : 0.10,
    'vainilla'  : 0.02,
    'chocolate' : 0.06,
}

### Visualización de la función de preferencias

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
t = np.arange(20, 40, 1)
t

In [ ]:
f = venta(t)
f

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('Temperatura')
plt.ylabel('Variación')
plt.title('Variación de venta como % contra la temperatura')
plt.grid(True)
plt.plot(t, f)

'''plt.annotate('Punto de Equilibrio', xy=(25,0), xytext=(23, 0.3),
             arrowprops=dict(fc='red', shrink=0.03, width=7,
                            headwidth=15, ec='red', connectionstyle="arc3, rad=-0.2")
            )'''

## Variación diaria de la temperatura

* La temperatura variará diariamente, pudiendo subir o bajar 2°C, hasta un mínimo de 22°C y un máximo de 39°C
* El precio del helado cambia de acuerdo a la temperatura prevista para el día del ajuste, hasta un 3% contra el precio base por cada grado de diferencia. El día más caliente, 40°C, el helado podría costar hasta 45% más
* El precio cambia todos los lunes

In [ ]:
def varprecio(t):
    p = precio_base * (1 + ((t - temperatura_base) * (random.uniform(0, 0.03))) )
    return p

In [ ]:
# Ejemplos de la variación del precio de acuerdo a la temperatura, con la variación máxima 3%

precio      = 50
temperatura = 40 # precio máximo

# Incremento máximo para la temperatura de 40°C
precio = precio * (1 + ((temperatura - temperatura_base) * (0.03)) )
print('El precio para este día será {0:0.2f}, una variación de {1:0.2f}% contra el precio base'.format(precio, 
                                                                                                    (precio / precio_base - 1)*100))

In [ ]:
# Ejemplo de variación con un número aleatorio
#
precio      = 50

# Incremento aleatorio de 3% máximo por cada grado de variación
precio = varprecio(temperatura)
print('El precio para este día será {0:0.2f}, una variación de {1:0.2f}% contra el precio base'.format(precio, 
                                                                                                    (precio / precio_base - 1)*100))

In [ ]:
# Variación de la temperatura día por día
temperatura      = 25

def vartemp(t):
    if random.randint(1,100) < 55:
        t -= 1
    else:
        t += 1
    
    
    t += random.randint(-1,1)
    
    if t < 22:
        t = 22

    if t > 40:
        t = 40
        
    return t
    
temperatura = vartemp(temperatura)
temperatura

### Gráfica de variación de la temperatura

Graficamos un intervalo de 90 días con sus cambios diarios de temperatura

In [ ]:
random.seed(123)

temperatura = 25
dias        = np.arange(0, 90, 1)
temps       = np.empty(0)

for i in range(0,90):
    temperatura = vartemp(temperatura)
    temps = np.append(temps, temperatura)

In [ ]:
temps

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(dias, temps)
plt.xlabel("Días")
plt.ylabel("Temperatura °C")
plt.title("Temperatura Diaria")

### Comportamiento de la venta

* La venta depende de la temperatura, siendo la base 25°C, para vender 100 helados
* Si la temperatura sube, el número de helados vendidos sube
* Se muestra la gráfica determinista y una gráfica con "ruido", que es la variación en la preferencia del consumidor para un día determinado

**Discusión** 

¿Cómo podríamos simular el comportamiento del consumidor con relación al precio?

In [ ]:
ventas  = np.empty(0)
ventasc = np.empty(0)
ventasf = np.empty(0)
ventasv = np.empty(0)
dias    = np.arange(0,90,1)

for i in range(0,90):
    v = round(100 * (1 + venta(temps[i])))
    ventas = np.append(ventas, v)
    ventasc = np.append(ventasc, v * 
                        random.uniform(1-sabores['chocolate'], 1+sabores['chocolate']))
    ventasf = np.append(ventasf, v * 
                        random.uniform(1-sabores['fresa'], 1+sabores['fresa']))
    ventasv = np.append(ventasv, v * 
                        random.uniform(1-sabores['vainilla'], 1+sabores['vainilla']))
    

plt.figure(figsize=(12,8))

plt.subplot(111)
plt.plot(dias, temps)
plt.title('Temperaturas por día')
plt.xlabel('Días')
plt.ylabel('Temperatura °C')

plt.figure(figsize=(12,8))
plt.subplot(111)
#plt.plot(dias,ventas,c='k',ls='-')
plt.plot(dias,ventasc,c='#00dd00',ls='--')
plt.plot(dias,ventasf,c='#dd0000',ls='--')
#plt.plot(dias,ventasv,c='#ffaa00',ls='--')
plt.title('Ventas por día')
plt.xlabel('Días')
plt.ylabel('Número de helados vendidos')


## Selección de dónde ocurrió la venta

Seleccionamos un lugar donde ocurrió la venta, que depende del día de la semana en el que estemos, por ejemplo, los miércoles (día 2) podemos vender en...

In [ ]:
localidad[2]

In [ ]:
random.choice(localidad[2])

## Cuántos helados se venden por lugar

* Calculamos que tanto afecta la venta, el que se venda en un lugar u otro
* El multiplicador afecta a la venta de todos los sabores por igual

In [ ]:
premium

In [ ]:
donde = 'playa'

mult = random.uniform(premium[donde]['min'], premium[donde]['max'])
print("multiplicador {0:0.4f}".format(mult) )

In [ ]:
def multiplicador(lugar):
    return random.uniform(premium[lugar]['min'], premium[lugar]['max'])

## Iteración (crear registros)


In [ ]:
df = pd.DataFrame(columns=['fecha', 'localidad', 'sabor', 'cant'
                           'temperatura', 'publicidad', 'precio'])
df

In [ ]:
precio           = 50
temperatura      = 25

In [ ]:
sabores

In [ ]:
from datetime import date, timedelta

fini  = date(2019, 6, 1)
ffin  = date(2019, 8, 30)
delta = timedelta(days=1)

df = pd.DataFrame(columns=['fecha', 'localidad', 'sabor', 'cant',
                           'temperatura', 'precio'])
while fini <= ffin:
    temperatura = vartemp(temperatura)
        
    # Actualizamos los precios si el día de la semana es 0 (lunes)
    if fini.weekday() == 0:
        precio = varprecio(temperatura)
        
    # En dónde se venden los helados
    donde = random.choice(localidad[fini.weekday()])
    
    # Calculamos la venta de helado de cada sabor
    #
    helado = {}

    for sabor in sabores:
        variabilidad = random.uniform(1-sabores[sabor], 1+sabores[sabor])
        cant = round(100 * (1 + venta(temperatura)) * variabilidad)
        helado[sabor] = cant
        df = df.append({'fecha': fini, 'temperatura': temperatura, 'precio': precio, 
                'localidad': donde, 'sabor': sabor, 'cant': cant}, ignore_index=True)
        
        print ("{0} {1}°C {2:0.2f} {3:7} {4:10} cant:{5:3}".format(fini.strftime("%Y-%m-%d"), 
                                               temperatura, precio,
                                               donde, sabor, cant))
    
    fini += delta

In [ ]:
df.head(5)

In [ ]:
df.to_excel('helados.xlsx', index=False, encoding='utf-8')